In [1]:
import sys
from utils import data_utils
import helper
import matplotlib.pyplot as plt
from utils import training_utils
from utils import data_utils
import torch
from model import models
import json
import os
from model import lightning_models
import math
import pytorch_lightning as pl


In [2]:
#config = helper.Config("./simulations",default_config_file="./default_configs/default_config_cifar10.ini")

config = helper.Config("./simulation_imagenet",default_config_file="./default_configs/default_config_imagenet1k.ini")

if config.INFO["fix_random_seed"]:
    pl.seed_everything(137) # To be reproducable
'''
# save the starting time as the last line
current_datetime,est_zone = helper.get_est_time_now()
if os.path.isfile(os.path.join("./simulation_imagenet","starting-time.txt")):
    with open(os.path.join("./simulation_imagenet","starting-time.txt"),"a") as f:
        f.write("\n")
        f.write(current_datetime.strftime("%Y-%m-%d %H:%M:%S"))
else:
    with open(os.path.join("./simulation_imagenet","starting-time.txt"),"a") as f:
        f.write(current_datetime.strftime("%Y-%m-%d %H:%M:%S"))
'''    

Seed set to 137


Loading default settings...
[INFO]
num_nodes = 1
gpus_per_node = 1
cpus_per_gpu = 8
precision = 16-mixed
fix_random_seed = True
strategy = auto

[DATA]
dataset = IMAGENET1K-5percent
n_views = 4
augmentations = ['RandomResizedCrop', 'GaussianBlur', 'RandomGrayscale', 'ColorJitter', 'RandomHorizontalFlip', 'RandomSolarize']
crop_size = 224
crop_min_scale = 0.08
crop_max_scale = 1.0
hflip_prob = 0.5
blur_kernel_size = 23
blur_prob = 1.0
grayscale_prob = 0.2
jitter_brightness = 0.4
jitter_contrast = 0.4
jitter_saturation = 0.2
jitter_hue = 0.1
jitter_prob = 0.8
solarize_prob = 0.0
imagenet_train_dir = /home/richard/Documents/code/clap/datasets/imagenet1k/train/train.lmdb
imagenet_val_dir = /home/richard/Documents/code/clap/datasets/imagenet1k/val/val.lmdb

[SSL]
backbone = resnet50
backbone_out_dim = 2048
use_projection_head = True
proj_dim = 4096
proj_out_dim = 256
optimizer = LARS
lr = 0.1
lr_scale = linear
grad_accumulation_steps = 1
momentum = 0.0
weight_decay = 0.1
lars_eta = 0.1
loss

'\n# save the starting time as the last line\ncurrent_datetime,est_zone = helper.get_est_time_now()\nif os.path.isfile(os.path.join("./simulation_imagenet","starting-time.txt")):\n    with open(os.path.join("./simulation_imagenet","starting-time.txt"),"a") as f:\n        f.write("\n")\n        f.write(current_datetime.strftime("%Y-%m-%d %H:%M:%S"))\nelse:\n    with open(os.path.join("./simulation_imagenet","starting-time.txt"),"a") as f:\n        f.write(current_datetime.strftime("%Y-%m-%d %H:%M:%S"))\n'

In [3]:
# for multi-gpu trainning, effective batch size = batch_size*num_gpus
ssl_batch_size = config.SSL["batch_size"] // (config.INFO["num_nodes"]*config.INFO["gpus_per_node"]*config.SSL["grad_accumulation_steps"])
ssl_train_loader,ssl_test_loader,ssl_val_loader = data_utils.get_dataloader(config.DATA,ssl_batch_size,num_workers=config.INFO["cpus_per_gpu"],
                                                                            validation=True,augment_val_set=True)
# test_loader and val_loader are not necessary
del ssl_test_loader
imgs,labels = next(iter(ssl_train_loader))
img_list, label_list = [],[]
for i_view in range(2):
    for j_img in range(2):
        img_list.append(imgs[i_view][j_img])
        #label_list.append(classes[labels[i_view][j_img]])
data_utils.show_images(img_list,2,2,label_list)
print(len(ssl_train_loader))
print(len(ssl_val_loader))

AttributeError: 'Config' object has no attribute 'SSl'

In [4]:
if config.SSL["lr_scale"] == "linear":
    ssl_lr = config.SSL["lr"]*config.SSL["batch_size"]/256.0 # lr ~ 0.1
elif config.SSL["lr_scale"] == "sqrt":
    ssl_lr = config.SSL["lr"]*math.sqrt(config.SSL["batch_size"]) # lr ~ 0.05
if "CIFAR" in config.DATA["dataset"] or "MNIST" in config.DATA["dataset"]:
    prune_backbone = True
else:
    prune_backbone = False
ssl_model = lightning_models.CLAP(backbone_name = config.SSL["backbone"],
                                  backbone_out_dim = config.SSL["backbone_out_dim"],
                                  prune = prune_backbone,
                                  use_projection_head=config.SSL["use_projection_head"],
                                  proj_dim = config.SSL["proj_dim"],
                                  proj_out_dim = config.SSL["proj_out_dim"],
                                  optim_name = config.SSL["optimizer"],
                                  lr = ssl_lr,
                                  momentum = config.SSL["momentum"],
                                  weight_decay = config.SSL["weight_decay"],
                                  eta = config.SSL["lars_eta"],
                                  warmup_epochs = config.SSL["warmup_epochs"],
                                  n_epochs = config.SSL["n_epochs"],
                                  n_views = config.DATA["n_views"],
                                  batch_size = config.SSL["batch_size"],
                                  lw0 = config.SSL["lw0"],
                                  lw1 = config.SSL["lw1"],
                                  lw2 = config.SSL["lw2"],
                                  rs = config.SSL["rs"],
                                  pot_pow = config.SSL["pot_pow"])

In [5]:
ssl_dir = os.path.join(config.loc,"ssl")
print("before making dir")
if not os.path.isdir(ssl_dir):
    os.makedirs(ssl_dir)
print("after making dir")
ssl_model = lightning_models.train_clap(model=ssl_model, 
                                        train_loader = ssl_train_loader,
                                        val_loader = ssl_val_loader,
                                        max_epochs=config.SSL["n_epochs"],
                                        every_n_epochs = config.SSL["save_every_n_epochs"],
                                        precision = config.INFO["precision"],
                                        checkpoint_path=ssl_dir)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


before making dir
after making dir
build logger
start to build the trainer
I am here
Found pretrained model at ./simulation_imagenet/ssl/best_val.ckpt, loading...


In [6]:
ssl_dir

'./simulation_imagenet/ssl'

In [7]:
lc_batch_size = config.LC["batch_size"] // (config.INFO["num_nodes"]*config.INFO["gpus_per_node"])
data_info = {"dataset":config.DATA["dataset"],"batch_size":lc_batch_size,"n_views":1,"augmentations":["RandomResizedCrop","RandomHorizontalFlip"],
            "crop_size":config.DATA["crop_size"],"crop_min_scale":0.08,"crop_max_scale":1.0,"hflip_prob":0.5}
# need to specify the location of the data for imagenet
if "IMAGENET1K" in config.DATA["dataset"]:
    data_info["imagenet_train_dir"] = config.DATA["imagenet_train_dir"]
    data_info["imagenet_val_dir"] = config.DATA["imagenet_val_dir"]

lc_train_loader,lc_test_loader,lc_val_loader = data_utils.get_dataloader(data_info,lc_batch_size,num_workers=config.INFO["cpus_per_gpu"],
                                                                         standardized_to_imagenet=config.LC["standardize_to_imagenet"])
# root directory for linear classification
lc_dir = os.path.join(config.loc,"lc")
if not os.path.isdir(lc_dir):
    os.makedirs(lc_dir)
if "lr_sweep" in config.LC:
    lr_list = config.LC["lr_sweep"]
else:
    lr_list = [config.LC["lr"]]
# sweep learning rates
best = {"best_test_acc1":0.0,"best_test_acc5":0.0,"best_test_loss":0.0,"best_model_dir":"none"}
for lr in lr_list:
    lc_sub_dir = os.path.join(lc_dir,"lr_{}".format(lr))
    os.makedirs(lc_sub_dir,exist_ok=True)
    if config.LC["lr_scale"] == "linear":
        lc_lr = lr*config.LC["batch_size"]/256.0 # lr ~ 0.1
    elif config.LC["lr_scale"] == "sqrt":
        lc_lr = lr*math.sqrt(config.LC["batch_size"]) # lr ~ 0.05
    # load the backbone from the check point
    best_ssl_ckpt = os.path.join(ssl_dir,"best_val.ckpt")
    ssl_model = lightning_models.CLAP.load_from_checkpoint(best_ssl_ckpt)
    ssl_model.backbone.remove_projection_head()

    lc_model = lightning_models.LinearClassification(
                 backbone = ssl_model.backbone,
                 in_dim = config.SSL["backbone_out_dim"],
                 out_dim = config.LC["output_dim"],
                 use_batch_norm = config.LC["use_batch_norm"],
                 optim_name = config.LC["optimizer"],
                 scheduler_name = config.LC["lr_scheduler"],
                 lr = lc_lr, 
                 momentum = config.LC["momentum"],
                 weight_decay = config.LC["weight_decay"],
                 n_epochs = config.LC["n_epochs"])
    
    lc_model = lightning_models.train_lc(linear_model = lc_model,
            train_loader = lc_train_loader,
            test_loader = lc_test_loader,
            val_loader = lc_val_loader,
            max_epochs = config.LC["n_epochs"],
            every_n_epochs = config.LC["save_every_n_epochs"],
            checkpoint_path = lc_sub_dir,
            precision = config.INFO["precision"],
            restart = config.LC["restart_training"])
    # get the best performed one
    with open(os.path.join(lc_sub_dir,"results.json")) as f:
        result = json.load(f)
    if result["test_acc1"] > best["best_test_acc1"]:
        best["best_test_acc1"] = result["test_acc1"] 
        best["best_test_acc5"] = result["test_acc5"] 
        best["best_test_loss"] = result["test_loss"]
        best["best_model_dir"] = lc_sub_dir
#save the information about the best model
with open(os.path.join(lc_dir,"results.json"),"w") as f:
    json.dump(best,f,indent=4)  
    

Found pretrained model at ./simulation_imagenet/lc/lr_0.1/best_val.ckpt, loading...
Found pretrained model at ./simulation_imagenet/lc/lr_0.2/best_val.ckpt, loading...
Found pretrained model at ./simulation_imagenet/lc/lr_0.3/best_val.ckpt, loading...


In [8]:
lc_dir

'./simulation_imagenet/lc'

In [9]:
# Fine-tune or semi-supervised learning
if len(config.SemiSL) > 0:
    semisl_batch_size = config.SemiSL["batch_size"] // (config.INFO["num_nodes"]*config.INFO["gpus_per_node"])
    for dataset in ["IMAGENET1K-1percent","IMAGENET1K-10percent"]:
        data_info = {"dataset":dataset,"batch_size":semisl_batch_size,"n_views":1,"augmentations":["RandomResizedCrop","RandomHorizontalFlip"],
                 "crop_size":config.DATA["crop_size"],"crop_min_scale":0.08,"crop_max_scale":1.0,"hflip_prob":0.5}
        # add the location for imagenet dataset
        data_info["imagenet_train_dir"] = config.DATA["imagenet_train_dir"]
        data_info["imagenet_val_dir"] = config.DATA["imagenet_val_dir"]
        
        semisl_train_loader,semisl_test_loader,semisl_val_loader = data_utils.get_dataloader(data_info,semisl_batch_size,num_workers=config.INFO["cpus_per_gpu"],
                                                                                 standardized_to_imagenet=config.SemiSL["standardize_to_imagenet"])
        semisl_dir = os.path.join(config.loc,"semisl-"+dataset)
        if not os.path.isdir(semisl_dir):
            os.makedirs(semisl_dir)
        if "lr_sweep" in config.SemiSL:
            lr_list = config.SemiSL["lr_sweep"]
        else:
            lr_list = [config.SemiSL["lr"]]
        # sweep learning rates
        best = {"best_test_acc1":0.0,"best_test_acc5":0.0,"best_test_loss":0.0,"best_model_dir":"none"}
        for lr in lr_list:
            semisl_sub_dir = os.path.join(semisl_dir,"lr_{}".format(lr))
            os.makedirs(semisl_sub_dir,exist_ok=True)
            if config.SemiSL["lr_scale"] == "linear":
                semisl_lr = lr*config.SemiSL["batch_size"]/256.0 # lr ~ 0.1
            elif config.SemiSL["lr_scale"] == "sqrt":
                semisl_lr = lr*math.sqrt(config.SemiSL["batch_size"]) # lr ~ 0.05
            # load the backbone from the checkpoint
            best_ssl_ckpt = os.path.join(ssl_dir,"best_val.ckpt")
            ssl_model = lightning_models.CLAP.load_from_checkpoint(best_ssl_ckpt)
            ssl_model.backbone.remove_projection_head()
            # load the best linear classifier from the checkpoint
            with open(os.path.join(lc_dir,"results.json")) as f:
                results = json.load(f)
                best_lc_dir = results["best_model_dir"] 
            lc_model = lightning_models.LinearClassification.load_from_checkpoint(os.path.join(best_lc_dir,"best_val.ckpt"),backbone = ssl_model.backbone)
            semisl_model = lightning_models.FineTune(backbone = ssl_model.backbone,
                    linear_net= lc_model.linear_net,
                    optim_name = config.SemiSL["optimizer"],
                    lr = semisl_lr, 
                    momentum = config.SemiSL["momentum"],
                    weight_decay = config.SemiSL["weight_decay"],
                    n_epochs = config.SemiSL["n_epochs"])
            semisl_model = lightning_models.train_finetune(
                    finetune_model = semisl_model,
                    train_loader = semisl_test_loader,
                    test_loader = semisl_test_loader,
                    val_loader = semisl_val_loader,
                    max_epochs = config.SemiSL["n_epochs"],
                    every_n_epochs = config.SemiSL["save_every_n_epochs"],
                    checkpoint_path = semisl_sub_dir,
                    precision= config.INFO["precision"],
                    restart = config.SemiSL["restart_training"])
            # get the best performed one
            with open(os.path.join(semisl_sub_dir,"results.json")) as f:
                result = json.load(f)
            if result["test_acc1"] > best["best_test_acc1"]:
                best["best_test_acc1"] = result["test_acc1"] 
                best["best_test_acc5"] = result["test_acc5"] 
                best["best_test_loss"] = result["test_loss"]
                best["best_model_dir"] = semisl_sub_dir
        #save the information about the best model
        with open(os.path.join(semisl_dir,"results.json"),"w") as f:
            json.dump(best,f,indent=4)  
            

Found pretrained model at ./simulation_imagenet/semisl-IMAGENET1K-1percent/lr_0.1/best_val.ckpt, loading...
Found pretrained model at ./simulation_imagenet/semisl-IMAGENET1K-1percent/lr_0.2/best_val.ckpt, loading...
Found pretrained model at ./simulation_imagenet/semisl-IMAGENET1K-1percent/lr_0.3/best_val.ckpt, loading...
Found pretrained model at ./simulation_imagenet/semisl-IMAGENET1K-10percent/lr_0.1/best_val.ckpt, loading...


Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4070 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/home/richard/miniconda3/envs/dl_env/lib/python3.12/site-packages/lightning_fabric/loggers/csv_logs.py:268: Experiment logs directory ./simulation_imagenet/semisl-IMAGENET1K-10percent/lr_0.2/logs/csv/version_0 exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
/home/richard/miniconda3/envs/dl_env/lib/python3.12/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /home/ri

Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

...

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |                                                | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     batch_test_acc1       0.001260403310880065
     batch_test_acc5       0.004921575076878071
     batch_test_loss         9.744197845458984
        test_acc1          0.001260403310880065
        test_acc5          0.004921575076878071
        test_loss            9.744195938110352
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [1]:
# Transfer learning(freeze backbone)
if len(config.TL) > 0:
    tl_output_dim = {"CIFAR100":100,
                    "FOOD101":101,
                    "FLOWERS102":102}
    for dataset in ["CIFAR100","FOOD101","FLOWERS102"]:
        tl_batch_size = config.TL["batch_size"] // (config.INFO["num_nodes"]*config.INFO["gpus_per_node"])
        data_info = {"dataset":dataset,"batch_size":semisl_batch_size,"n_views":1,"augmentations":["RandomResizedCrop","RandomHorizontalFlip"],
                 "crop_size":config.DATA["crop_size"],"crop_min_scale":0.08,"crop_max_scale":1.0,"hflip_prob":0.5}
        tl_train_loader,tl_test_loader,tl_val_loader = data_utils.get_dataloader(data_info,lc_batch_size,num_workers=config.INFO["cpus_per_gpu"],
                                                                                 standardized_to_imagenet=config.TL["standardize_to_imagenet"])
        tl_dir = os.path.join(config.loc,"tl-"+dataset)
        if not os.path.isdir(tl_dir):
            os.makedirs(tl_dir)
        if "lr_sweep" in config.SemiSL:
            lr_list = config.SemiSL["lr_sweep"]
        else:
            lr_list = [config.SemiSL["lr"]]
        # sweep learning rates
        best = {"best_test_acc1":0.0,"best_test_acc5":0.0,"best_test_loss":0.0,"best_model_dir":"none"}
        for lr in lr_list:
            tl_sub_dir = os.path.join(tl_dir,"lr_{}".format(lr))
            os.makedirs(tl_sub_dir,exist_ok=True)
            if config.TL["lr_scale"] == "linear":
                tl_lr = lr*config.TL["batch_size"]/256.0 # lr ~ 0.1
            elif config.TL["lr_scale"] == "sqrt":
                tl_lr = lr*math.sqrt(config.TL["batch_size"]) # lr ~ 0.05
            # load the backbone from the checkpoint
            best_ssl_ckpt = os.path.join(ssl_dir,"best_val.ckpt")
            ssl_model = lightning_models.CLAP.load_from_checkpoint(best_ssl_ckpt)
            ssl_model.backbone.remove_projection_head()
        
            tl_model = lightning_models.LinearClassification(
                    backbone = ssl_model.backbone,
                    in_dim = config.SSL["backbone_out_dim"],
                    out_dim = tl_output_dim[dataset],
                    use_batch_norm = config.TL["use_batch_norm"],
                    optim_name = config.TL["optimizer"],
                    lr = tl_lr, 
                    scheduler_name= config.TL["lr_scheduler"],
                    momentum = config.TL["momentum"],
                    weight_decay = config.TL["weight_decay"],
                    n_epochs = config.TL["n_epochs"])

            tl_model = lightning_models.train_lc(
                    linear_model = tl_model,
                    train_loader = tl_train_loader,
                    val_loader = tl_val_loader,
                    test_loader = tl_test_loader,
                    every_n_epochs = config.TL["save_every_n_epochs"],
                    max_epochs = config.TL["n_epochs"],
                    precision = config.INFO["precision"],
                    checkpoint_path = tl_sub_dir,
                    restart = config.LC["restart_training"]) 
                        # get the best performed one
            with open(os.path.join(tl_sub_dir,"results.json")) as f:
                result = json.load(f)
            if result["test_acc1"] > best["best_test_acc1"]:
                best["best_test_acc1"] = result["test_acc1"] 
                best["best_test_acc5"] = result["test_acc5"] 
                best["best_test_loss"] = result["test_loss"]
        #save the information about the best model
        with open(os.path.join(tl_dir,"results.json"),"w") as f:
            json.dump(best,f,indent=4)  

NameError: name 'config' is not defined